In [153]:

# libraries
import random
import pandas as pd
import snoop
import feather
import time
import gc
import sys
import feather
import os
import joblib
import numpy as np

from varname import nameof
from icecream import ic

from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [154]:
# Load the model from the file
clf_from_joblib = joblib.load('model_mlpclassifier.pkl')
clf_from_joblib

MLPClassifier(max_iter=1000, random_state=19)

In [155]:
# generate the fake dataset.
nrows = 200

def make_database(index):
    #passengerid = index
    pclass    = int(random.randint(1, 3))
    #name      = int(random.randint(0, 1306))
    sex       = int(random.randint(0, 1))
    #age       = int(random.randint(2, 80))
    #sibsp     = int(random.randint(0, 8))
    #parch     = int(random.randint(0, 9))
    #ticket    = int(random.randint(0, 928))
    fare      = int(round(random.uniform(0, 513), 4))
    #cabin     = int(random.randint(0, 185))
    embarked  = int(random.randint(0, 2))
    boat      = int(random.randint(0, 26))
    #body      = int(round(random.uniform(0, 328), 2))
    #home_dest = int(random.randint(0,368))
    survived  = int(random.randint(0, 1))
    
    # 'nameof' get name of variables and are appended in one list with column names.
    name_cols = [] # list
    #name_cols.append(nameof(passengerid))
    name_cols.append(nameof(pclass))
    #name_cols.append(nameof(name))
    name_cols.append(nameof(sex))
    #name_cols.append(nameof(age))
    #name_cols.append(nameof(sibsp))
    #name_cols.append(nameof(parch))
    #name_cols.append(nameof(ticket))
    name_cols.append(nameof(fare))
    #name_cols.append(nameof(cabin))
    name_cols.append(nameof(embarked))
    name_cols.append(nameof(boat))
    #name_cols.append(nameof(body))
    #name_cols.append(nameof(home_dest))
    name_cols.append(nameof(survived))
    
    # fake database
    database = {#f'passengerid': passengerid,
                f'pclass': pclass,
                #f'name': name,
                f'sex': sex,
                #f'age': age,
                #f'sibsp': sibsp,
                #f'parch': parch,
                #f'ticket': ticket,
                f'fare': fare,
                #f'cabin': cabin,
                f'embarked': embarked,
                f'boat': boat,
                #f'body': body,
                #f'home_dest': home_dest,
                f'survived': survived
               }
    
    return name_cols, database

def generate_fakedata(nrows):    
    
    irow = 1
    index = 1310
    name_cols, database = make_database(index)
    df = pd.DataFrame(columns=name_cols)
        
    while(irow <= nrows):
        name_cols, database = make_database(index)
        df_database = pd.DataFrame([database])
        df = pd.concat([df, df_database], ignore_index=True)
        irow += 1
        index += 1

    return df

In [156]:
df = generate_fakedata(nrows)

In [157]:
df

,pclass,sex,fare,embarked,boat,survived
0,1,1,423,1,22,0
1,2,1,27,1,18,0
2,2,1,453,1,19,1
3,3,1,85,2,16,0
4,2,1,454,1,20,1
...,...,...,...,...,...,...
195,3,1,364,0,3,0
196,3,1,485,1,10,0
197,3,1,57,2,19,0
198,1,1,84,2,23,1


In [158]:
# convert object type to category type
df[df.select_dtypes(['object']).columns] = df.select_dtypes(['object']).apply(lambda x: x.astype('int64'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   pclass    200 non-null    int64
 1   sex       200 non-null    int64
 2   fare      200 non-null    int64
 3   embarked  200 non-null    int64
 4   boat      200 non-null    int64
 5   survived  200 non-null    int64
dtypes: int64(6)
memory usage: 9.5 KB


In [159]:
# Alternatively X and y can be obtained directly from the dataframe attribute:
X = df.drop('survived', axis=1) # drop the column target
y = df['survived'] # target

In [160]:
kf=KFold(n_splits=3)
score=cross_val_score(clf_from_joblib,X,y,cv=kf)

In [161]:
print("Cross Validation Scores are {}".format(score))
print("Average Cross Validation score :{}".format(score.mean()))

Cross Validation Scores are [0.41791045 0.6119403  0.48484848]
Average Cross Validation score :0.5048997437057139


In [162]:
df.columns

Index(['pclass', 'sex', 'fare', 'embarked', 'boat', 'survived'], dtype='object')

In [163]:
## Anderson: sex: 1| pclass: 3| fare: 150| embarked: 2 |  boat: 2
anderson = np.array([3,1,150,2,3]).reshape(1,-1)
print('anderson'.format(clf_from_joblib.predict(anderson[0])))

NotFittedError: This MLPClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.